In [1]:
from osgeo import gdal
import numpy as np
import pandas as pd

In [2]:
img_path = '/kaggle/input/tdbb-map/LC8_2020_UTM.tif'

ds = gdal.Open(img_path, gdal.GA_ReadOnly)

data = ds.ReadAsArray()

bands, rows, cols = data.shape
rsl = []
for i in range(rows):
    for j in range(cols):
        if data[0, i, j] > -9999:
            tmp = np.append([i, j], data[:, i, j])
            rsl.append(tmp)

df = pd.DataFrame(rsl, columns=['rows', 'cols', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19'])
df = df.astype({'rows': 'int', 'cols': 'int'}) # ép kiểu int
df.drop(['B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19'], axis = 1, inplace=True)
df.to_csv('map.csv', index=False)

In [4]:
df

,rows,cols,B1,B2,B3,B4,B5,B6,B7
0,8,1835,0.011420,0.015160,0.043127,0.029378,0.272670,0.126205,0.047995
1,8,1836,0.015627,0.018652,0.050360,0.028553,0.372963,0.147655,0.058005
2,8,1837,0.016232,0.018955,0.050635,0.026325,0.418970,0.145703,0.056410
3,8,1841,0.020632,0.027425,0.065485,0.042000,0.443005,0.164073,0.072663
4,9,1835,0.020742,0.023685,0.056767,0.047335,0.313947,0.180793,0.081985
...,...,...,...,...,...,...,...,...,...
3988516,3135,1749,0.026930,0.027865,0.054265,0.028938,0.295963,0.159260,0.064110
3988517,3135,1750,0.025693,0.026655,0.048765,0.027177,0.284523,0.125132,0.049122
3988518,3135,1751,0.017085,0.024152,0.044420,0.029130,0.270608,0.115067,0.047775
3988519,3135,1752,0.020385,0.028800,0.056190,0.035977,0.273523,0.153568,0.068675


In [7]:
raster_df = pd.DataFrame({'x': df['rows'], 'y': df['cols']})

In [8]:
raster_df

,x,y
0,8,1835
1,8,1836
2,8,1837
3,8,1841
4,9,1835
...,...,...
3988516,3135,1749
3988517,3135,1750
3988518,3135,1751
3988519,3135,1752


In [9]:
img_path = '/kaggle/input/tdbb-map/LC8_2020_UTM.tif'
data_path = '/kaggle/input/select-points/data.csv'

ds = gdal.Open(img_path)

data = ds.ReadAsArray()
geotransform = ds.GetGeoTransform()

bands, rows, cols = data.shape

In [10]:
df_data = pd.read_csv(data_path)

pixel_coords = []
coords = list(zip(df_data['X'], df_data['Y']))

# Lặp qua từng tọa độ trong danh sách
for x_coord, y_coord in coords:
  # Tính toán vị trí điểm ảnh
  x_pixel = int((x_coord - geotransform[0]) / geotransform[1])
  y_pixel = int((y_coord - geotransform[3]) / geotransform[5])

  # Kiểm tra xem điểm ảnh có nằm trong giới hạn của ảnh hay không
  if 0 <= x_pixel < ds.RasterXSize and 0 <= y_pixel < ds.RasterYSize:
        pixel_coords.append((x_pixel, y_pixel))
  else:
        pixel_coords.append((-9999, -9999))  # Assign a default value

pixel_coords_df = pd.DataFrame(pixel_coords, columns=['X', 'Y'])
df_data[['X', 'Y']] = pixel_coords_df[['X', 'Y']]

values = []

num_bands = ds.RasterCount

for x_pixel, y_pixel in pixel_coords:
  band_values = []

  for band_num in range(1, num_bands + 1):
      band = ds.GetRasterBand(band_num)
      value = band.ReadAsArray(x_pixel, y_pixel, 1, 1)[0, 0]  # Đọc giá trị tại điểm ảnh
      band_values.append(value)

  values.append(band_values)

  values_array = np.array(values)

  band_columns = [f'B{i}' for i in range(1, num_bands + 1)]

  # Thêm các giá trị từ TIFF vào DataFrame
df_data[band_columns] = values_array

df_data.to_csv('data.csv', index=False)
df_data.drop(['B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19'], axis = 1, inplace=True)

In [11]:
df_data

,X,Y,Label,name,B1,B2,B3,B4,B5,B6,B7
0,833,896,18,Open_water,0.062570,0.069720,0.112372,0.129120,0.077062,0.033997,0.026023
1,1907,1287,18,Open_water,0.001121,0.012589,0.049975,0.030546,0.033874,0.025734,0.021636
2,1215,822,18,Open_water,0.049837,0.059586,0.109980,0.102954,0.053825,0.018969,0.011750
3,711,1275,18,Open_water,0.026930,0.032183,0.063670,0.039662,0.034740,0.018322,0.012520
4,1922,1107,18,Open_water,0.049673,0.069390,0.122080,0.107725,0.077777,0.039662,0.026682
...,...,...,...,...,...,...,...,...,...,...,...
895,2251,1307,19,Aquaculture,0.034162,0.048022,0.087430,0.075000,0.173450,0.076815,0.044997
896,1598,587,19,Aquaculture,0.040130,0.049521,0.092160,0.084061,0.140794,0.079716,0.045520
897,1601,1107,19,Aquaculture,0.025569,0.032650,0.077118,0.049755,0.357879,0.144767,0.068400
898,1917,1243,19,Aquaculture,0.050277,0.051652,0.077365,0.058638,0.314855,0.206450,0.094635


In [12]:
def get_neighbor_values(row, col):
    # Create an empty 3x3x7 array filled with zeros
    neighbor_values = np.zeros((3, 3, 7))
    for i in range(row - 1, row + 2):
        for j in range(col - 1, col + 2):
            # Check if the current neighbor is within the image bounds and not the center pixel
            if (i != row or j != col) and 0 <= i < ds.RasterYSize and 0 <= j < ds.RasterXSize:
                pixel_values = []
                for band_num in range(1, 8):
                    band = ds.GetRasterBand(band_num)
                    value = band.ReadAsArray(j, i, 1, 1)[0, 0]
                    pixel_values.append(value)
                # Calculate the index within the 3x3x7 array
                row_index = i - (row - 1)
                col_index = j - (col - 1)
                neighbor_values[row_index, col_index] = pixel_values
    return neighbor_values



In [13]:
df['neighbor_values'] = df_data.apply(lambda row: get_neighbor_values(row['X'], row['Y']), axis=1)
#combine the list of 3x3x7 arrays into a single array.
# filter out rows with invalid coordinates
valid_rows = df_data[(df_data['X'] != -9999) & (df_data['Y'] != -9999)]
data = np.stack(df.loc[valid_rows.index, 'neighbor_values'].to_numpy())

# Filter out labels corresponding to invalid coordinates
# Convert labels to a NumPy array, force type int
labels = df_data.loc[valid_rows.index, 'Label'].to_numpy().astype(int)

for i in range(len(labels)):
    if labels[i] == 10:
        labels[i] = 2
    elif labels[i] == 18:
        labels[i] = 5
    elif labels[i] == 19:
        labels[i] = 6

In [14]:
import numpy as np

# Check for NaN in 'data'
print(np.isnan(data).any())

# Check for infinite values in 'data'
print(np.isinf(data).any())

# Replace NaN with 0 in 'data' (consider other strategies like imputation)
data = np.nan_to_num(data, nan=0.0)

True
False


In [15]:
data

array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]],


       [[[0.0316875 , 0.04519   , 0.0822875 , ..., 0.179775  ,
          0.0735425 , 0.036665  ],
         [0.

In [16]:
labels

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4,

In [17]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(3, 3, 7), padding='same'), # add padding='same'
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1229 - loss: 2.2870
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0986 - loss: 2.2608 
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1243 - loss: 2.2440
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1574 - loss: 2.2265
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1594 - loss: 2.2087 
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1656 - loss: 2.1817
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1788 - loss: 2.1844 
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1733 - loss: 2.1632
Epoch 9/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1663 - loss: 2.1782
Epoch 10/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1869 - loss: 2.1515
Epoch 11/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1858 - loss: 2.1568
Epoch 12/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1498 - loss: 2.16

In [18]:
raster_data = []
for index, row in raster_df.iterrows():
    x, y = row['x'], row['y']
    neighbor_values = []
    for i in range(x - 1, x + 2):
        for j in range(y - 1, y + 2):
            if 0 <= i < ds.RasterXSize and 0 <= j < ds.RasterYSize:
                pixel_values = [ds.GetRasterBand(k).ReadAsArray(i, j, 1, 1)[0, 0] for k in range(1, 8)]
                neighbor_values.extend(pixel_values)
            else:
                neighbor_values.extend([0] * 7)  # Padding nếu điểm ảnh nằm ngoài ảnh
    raster_data.append(neighbor_values)
raster_data = np.array(raster_data).reshape(-1, 3, 3, 7)


In [22]:
predictions = model.predict(raster_data)

124642/124642 ━━━━━━━━━━━━━━━━━━━━ 179s 1ms/step


In [20]:
predicted_classes = np.argmax(predictions, axis=1)
output_map = predicted_classes.reshape(rows, cols)

ValueError: cannot reshape array of size 3988521 into shape (3142,2635)

In [25]:
output_df = raster_df[['x', 'y']].copy()  

output_df['predicted_class'] = predicted_classes 

print(output_df)

            x     y  predicted_class
0           8  1835                9
1           8  1836                9
2           8  1837                9
3           8  1841                9
4           9  1835                9
...       ...   ...              ...
3988516  3135  1749                7
3988517  3135  1750                7
3988518  3135  1751                7
3988519  3135  1752                7
3988520  3135  1753                7

[3988521 rows x 3 columns]


In [26]:
ds_output = gdal.Open(img_path, gdal.GA_ReadOnly)
data_output = ds_output.ReadAsArray()
bands, rows, cols = data_output.shape
output_map = np.zeros(shape=(rows, cols), dtype=np.float32) + 255
for lines in output_df.values:
  i, j, pred = lines
  output_map[int(i)][int(j)] = pred

In [27]:
output_map

array([[255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       ...,
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.]], dtype=float32)

In [ ]:
outfname = 'LCmap.output.tif'
driver = gdal.GetDriverByName("GTiff")

dst_ds = driver.Create(outfname, cols, rows, 1, gdal.GDT_Float32)
dst_ds.SetGeoTransform(ds.GetGeoTransform())
dst_ds.SetProjection(ds.GetProjection())

band = dst_ds.GetRasterBand(1)
band.SetNoDataValue(255)
band.WriteArray(output_map)
dst_ds = None

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data = scaler.fit_transform(data.reshape(-1, data.shape[-1])).reshape(data.shape)  # Reshape for scaling